# Data Generation

In [1]:
import sys
from pathlib import Path


project_root = Path().resolve().parent
sys.path.append(str(project_root / "src"))

In [2]:
from data_generation import generate_omnichannel_orders


df = generate_omnichannel_orders()

In [3]:
raw_dir = project_root / "data" / "raw"
raw_dir.mkdir(parents=True, exist_ok=True)

df.to_csv(
    raw_dir / "omnichannel_orders.csv",
    index=False
)

# Experiment Design

In [4]:
from experiment_design import assign_experiment_groups

df_exp = assign_experiment_groups(df)

processed_dir = project_root / "data" / "processed"
processed_dir.mkdir(parents=True, exist_ok=True)

df_exp.to_csv(
    processed_dir / "experiment_data.csv",
    index=False
)

# Analysis

In [5]:
from analysis import ab_test
import pandas as pd


metrics = ["on_time", "fulfillment_cost", "guest_satisfaction"]

results = {
    m: ab_test(df_exp, m)
    for m in metrics
}

pd.DataFrame(results).T

,control_mean,treatment_mean,lift,p_value
on_time,0.357223,0.434119,0.076896,2.150509e-69
fulfillment_cost,7.607059,7.399428,-0.207631,1.043710e-72
guest_satisfaction,4.607602,4.608776,0.001174,8.325745e-01


# KPIs

In [6]:
from kpis import compute_exec_kpis


exec_view = (
    df_exp
    .groupby("experiment_group", group_keys=False)
    .apply(compute_exec_kpis, include_groups=False)
)

exec_df = pd.DataFrame(exec_view.tolist(), index=exec_view.index)
exec_df

,on_time_rate,avg_cost,avg_satisfaction,orders
experiment_group,,,,
control,0.357223,7.607059,4.607602,25046
treatment,0.434119,7.399428,4.608776,24954
